# Compétition: Natural Language Processing with Disaster Tweets

## Analyse des jeux de données

* sample_submission est le jeu contenant les réponses
* train.csv est le jeu d'entraînement avec le label
* test.csv est le jeu de test sur lequel faire tourner le modèle

### Sample_submission

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from transformers import XLNetTokenizer, TFXLNetModel, \
                        RobertaTokenizer, TFRobertaModel

from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow_addons.metrics import F1Score

from tensorflow.keras.backend import clear_session
# import keras.backend as K
from numba import cuda
import gc
import keras_tuner as kt
from time import sleep

In [ ]:
sample_sub = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_sub.head(10)

Les résultats de la prédiction pour la compétition doivent être retournée dans un tableau de n x 2 dimensions avec:
* id: la colonne contenant les "id" des tweets
* target: lalabel prédit par mon modèle

### Train

In [ ]:
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
train_df.head(50)

In [ ]:
train_df.info()

Le jeu est composé de 7613 individus pour 5 features.

"keyword" et "location" possèdent des valeurs nulles. Analysons les.

In [ ]:
cols = []
nb_val_null = []

for col in train_df.columns:
    cols.append(col)
    nb_val_null.append(train_df[col].isnull().sum())

for col, nb_null in zip(cols, nb_val_null):
    print(f"{col}: {nb_null} valeurs nulles, soit {round(nb_null / len(train_df) *100, 3)}%")

plt.bar(cols, nb_val_null, 0.8)
plt.ylim(0, len(train_df))

train_quant = np.quantile([0, len(train_df)], [0.25, 0.5])

plt.axhline(train_quant[0], color= "g", linestyle= "--", label= "Quartile 25%")
plt.axhline(train_quant[1], color= "r", linestyle= "--", label= "Quartile 50%")

plt.show()

La feature "location" contient plus de 33% de valeurs nulles. De plus, ces valeurs étant des variables nominales; il m'est quasiment impossible de les compléter. Je vais donc droper cette colonne.

"keyword" posséde 0.8% de valeurs nulles, je vais essayer d'entraîner une classification pour combler les valeurs manquantes.


In [ ]:
print(f"Il y a {train_df['keyword'].nunique()} valeurs uniques dans la feature 'keyword'")
train_df["keyword"].unique()

On peut voir que certains keywords possèdent plusieurs mots séparés par un espace encodé (%20). Il sera donc potentiellement nécessaire d'effectuer un remplacement. Je vais utilise le méthode python urllib.parse.unquote().


### Test

In [ ]:
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")
test_df[test_df["location"].notnull()].head(10)

"location" contient le même genre de valeurs que son homologue dans le jeu d'entraînement. La suppression de cette colonne est confirmée.

In [ ]:
test_df.info()

Le jeu est composé de 4 features pour 3263 individus.

"keyword" et "location" possèdent des valeurs nulles.

In [ ]:
cols = []
nb_val_null = []

for col in test_df.columns:
    cols.append(col)
    nb_val_null.append(test_df[col].isnull().sum())

for col, nb_null in zip(cols, nb_val_null):
    print(f"{col}: {nb_null} valeurs nulles, soir {round(nb_null / len(test_df) *100, 3)}%")

plt.bar(cols, nb_val_null, 0.8)
plt.ylim(0, len(train_df))

test_quant = np.quantile([0, len(test_df)], [0.25, 0.5])

plt.axhline(test_quant[0], color= "g", linestyle= "--", label= "Quartile 25%")
plt.axhline(test_quant[1], color= "r", linestyle= "--", label= "Quartile 50%")

plt.show()

Dans ce jeu aussi, je vais utliser une concaténation des jeux d'entraînement et de test pour avoir un corpus plus grand pour compléter les valeurs nulles de "keyword".

In [ ]:
print(f"Il y a {test_df['keyword'].nunique()} valeurs uniques dans la feature 'keyword'")
test_df["keyword"].unique()

In [ ]:
assert train_df["keyword"].unique().tolist() == test_df["keyword"].unique().tolist()

Les valeurs composant la feature "keyword" sont identiques.

Analysons les.

In [ ]:
train_val_hist = train_df["keyword"].value_counts().reset_index()
train_val_hist.sort_values(by= "index", inplace= True)

In [ ]:
fig = plt.figure(figsize= (25, 10))

plt.bar(train_val_hist.loc[:, "index"], round(
    train_val_hist.loc[:, "keyword"] / train_val_hist["keyword"].max() *100, 1))
plt.axhline(np.mean(round(
    train_val_hist.loc[:, "keyword"] / train_val_hist["keyword"].max() *100, 1)), 
    color= "r", linestyle= "--")

plt.show()

In [ ]:
test_val_hist = test_df["keyword"].value_counts().reset_index()
test_val_hist.sort_values(by= "index", inplace= True)

In [ ]:
fig = plt.figure(figsize= (25, 10))

plt.bar(test_val_hist.loc[:, "index"], round(
    test_val_hist.loc[:, "keyword"] / test_val_hist["keyword"].max() *100, 1))
plt.axhline(np.mean(round(
    test_val_hist.loc[:, "keyword"] / test_val_hist["keyword"].max() *100, 1)), 
    color= "r", linestyle= "--")

plt.show()

On voit clairement  que les mots de la feature "keyword" ne sont pas répartis de la même manière dans le jeu d'entraînement que dans le jeu de test.

Le nombre de "keyword" du jeu d'entraînment sont relativement proche de la moyenne alors qu'ils sont plus aléatoirement répartis dans le jeu de test.


### Mise en forme des jeux de données

* [x] Retrait de la colonne "location"
* [x] Retrait des liens
* Tokenisation de la colonne "text"
  * [x] Retrait des citations des noms d'utilisateurs
  * [x] Garde des hashtags! et de toutes les combinaisons alphanumériques
  * [x] Doublon des hashtags sans le symbole '#'
* [x] Lemmatisation de la colonne "text"

In [ ]:
train_dff = train_df.drop("location", axis= 1)
test_dff = test_df.drop("location", axis= 1)


In [ ]:
def remove_url(text):
    text_f = []
    for word in text.split():
        word = word.lower()
        word = re.sub(r'https?:\/\/.*', "", word)
        word = re.sub(r'^@.*', "", word)
        word = re.sub(r'\&amp', "&", word)
        
        text_f.append(word)
        
        if re.search(r'^#.*', word) is not None:
            text_f.append(re.split(r'^#', word)[-1])

    return " ".join(text_f)


train_dff["text"] = train_dff["text"].apply(lambda x: remove_url(x))
test_dff["text"] = test_dff["text"].apply(lambda x: remove_url(x))

In [ ]:
train_dff["text"].head(25)

In [ ]:
stopwords = set(stopwords.words("english"))
regex_form = r'#?[a-zA-Z0-9]+|\&'
tokenizer = RegexpTokenizer(regex_form)

def tokenisation_stopwords(text, tokenizer, stopwords):
    words = tokenizer.tokenize(text)
    words = [word for word in words if word not in stopwords]

    return words


In [ ]:
train_dff["text"] = train_dff["text"].apply(lambda x: tokenisation_stopwords(x, tokenizer, stopwords))
test_dff["text"] = test_dff["text"].apply(lambda x: tokenisation_stopwords(x, tokenizer, stopwords))

In [ ]:
train_dff.loc[12, "text"]

In [ ]:
lematizer = WordNetLemmatizer()

train_dff["text"] = train_dff["text"].apply(lambda x: [lematizer.lemmatize(word, "n") for word in x])
train_dff["text"] = train_dff["text"].apply(lambda x: [lematizer.lemmatize(word, "v") for word in x])

test_dff["text"] = test_dff["text"].apply(lambda x: [lematizer.lemmatize(word, "n") for word in x])
test_dff["text"] = test_dff["text"].apply(lambda x: [lematizer.lemmatize(word, "v") for word in x])

In [ ]:
train_dff.loc[12, "text"]

### Transformation des données de la feature "keyword"

L'idée était de calculer un poids à partir de la feature "keyword" et de Word2Vec. Après ajout de ce poids dans le texte et entraînement avec XLNet et RoBERTa les résultats des métriques sont les mêmes. Cette étape est donc retirée.

~~Je vais entraîner les tweets avec word2vec. Cela va permettre de récupérer les coefficients de similarité entre le mot dans la feature "keyword" et le tweet et de calculer le poids de "keyword" par rapport au tweet.~~

~~Si "keyword" == null, le poids est fixé à 0~~

~~Si "keyword" == 1+ mot(s), le poids est la somme des coefficients de similarité présents sur x mots (x est le nombre de mots retourné par la méthode "most_similar"; à définir)~~


Par contre, je peux ajouter ajouter les "keyword" désencodés au tweets.

In [ ]:
from urllib.parse import unquote

def unquote_join(keyword, text):
    if keyword is np.nan:
        return " ".join(text)
    else:
        tmp_keyword = unquote(keyword).split()
        return " ".join(text + tmp_keyword)


train_dff["decoded_joined"] = train_dff[["keyword", "text"]].apply(lambda t: 
                                                                   unquote_join(t[0], t[1]), axis= 1)
test_dff["decoded_joined"] = test_dff[["keyword", "text"]].apply(lambda t: 
                                                                   unquote_join(t[0], t[1]), axis= 1)

### Répartition des mots dans les tweets

In [ ]:
def repartition_mots(df, df_name):
    tmp_len = []
    for i in range(len(df)):
        tmp_len.append(len(df.loc[i, "decoded_joined"].split()))

    print(f"\nJeu: {df_name}\n{'*' *50}\n")
    print(f"Nb mini de mots: {min(tmp_len)}")
    print(f"Nb maxi de mots: {max(tmp_len)}")
    nb_mots_max = max(tmp_len)
    print(f"Nb moyenne de mots: {np.mean(tmp_len)}")
    print(f"Nb de mots médian: {np.quantile(tmp_len, 0.5)}")

    fig = plt.figure(figsize= (25, 10))
    plt.hist(tmp_len, bins= max(tmp_len))
    plt.axvline(np.mean(tmp_len), color= "r", linestyle= "--", label= "Moyenne")
    plt.axvline(np.quantile(tmp_len, 0.5), color= "g", linestyle= "--", label= "Médiane")
    plt.xlim(min(tmp_len), max(tmp_len))

    plt.legend()

    plt.show()

    return nb_mots_max


In [ ]:
nb_mots_max = []

nb_mots_max.append(repartition_mots(train_dff, "Train"))
nb_mots_max.append(repartition_mots(test_dff, "Test"))

nb_mots_max = np.max(nb_mots_max)

Le nombre maximum de mots dans un tweet est de 30 mots.

Je vais maintenant afficher un nuage de mots pour les jeux d'entraînement et de test.


In [ ]:
def prepare_cloud(df):
    comment_words = ""

    for i in range(len(df)):
        comment_words += df.loc[i, "decoded_joined"] + " "

        if (i +1) % 1000 == 0:
            print(f"{i +1}/{len(df)}")
    
    return comment_words


In [ ]:
from wordcloud import WordCloud, STOPWORDS

stopwords = set(STOPWORDS)

w_cloud_train = WordCloud(width= 1600, height= 1600, background_color= "white", 
    stopwords= stopwords, min_font_size= 8).generate(prepare_cloud(train_dff))
w_cloud_test = WordCloud(width= 1600, height= 1600, background_color= "white", 
    stopwords= stopwords, min_font_size= 8).generate(prepare_cloud(test_dff))

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(w_cloud_train)
plt.title("Nuage de mots du jeu Train")

plt.show()

fig = plt.figure(figsize=(10, 10))
plt.imshow(w_cloud_test)
plt.title("Nuage de mots du jeu Test")

plt.show()

Sur le jeu d'entraînement, on peut voir que les mots "go", "u", "one", "new", "fire" sont les plus présents.

Sur le jeu de test, on peut voir que les mots "go" et "new", "say", "fire", "one", "u" sont les plus présents.



# XLNet

Je vais chercher une méthode de classification rapide et efficace. paperswithcode.com est une piste à suivre.

Je vais utiliser la méthode XLNet de la librairie "transformers". En effet, ce modèle est ce qu'il se fait de mieux pour le moment en ce qui concerne [la classification de textes](https://paperswithcode.com/sota/text-classification-on-ag-news)

D'abord, je vais créer les jeux de train et de test. Pour le jeu de validation, j'utiliserai le paramètre "validation_split" de la méthode fit().

In [ ]:
# Séparation de l'ensemble train/validation du jeu de test
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(train_dff["decoded_joined"], 
    train_dff["target"], train_size= 0.8, random_state= 42)

print("X: ", X_train_df.iloc[:10])
print(f"y: {y_train_df.iloc[:10]}")


In [ ]:
def remove_empty(df_train, df_test):
    tmp_df_train = df_train.copy()
    tmp_df_test = df_test.copy()
    for idx in df_train.index:
        if len(df_train[idx]) == 0:
            tmp_df_train = tmp_df_train.drop(idx)
            tmp_df_test = tmp_df_test.drop(idx)
    
    return tmp_df_train, tmp_df_test


In [ ]:
X_train_dff, y_train_dff = remove_empty(X_train_df, y_train_df)
X_test_dff, y_test_dff = remove_empty(X_test_df, y_test_df)

XLNet est fourni avec des modèles pré-entraînés pour effectuer du partial fine tuning.
Les modèles sont trouvables [ici](https://huggingface.co/models?library=tf&sort=modified&search=xlnet)

J'ai choisi "xlnet_large_cased" qui est le plus gros jeu pour des textes en anglais et le plus récent.

D'ailleurs, je suis tombé sur ["riccardode/tweets_disaster"](https://huggingface.co/riccardode/tweets_disaster) qui semble plus approprié. Ce dernier sera étudié par la suite.

In [ ]:
# Modèle XLNet pré-entraîné
xlnet_pretrained_model = "xlnet-large-cased"

# Chargement du tokenizer avec les poids du modèle choisi
xlnet_tokenizer = XLNetTokenizer.from_pretrained(xlnet_pretrained_model)


Tokenisation des tweets

In [ ]:
X_train_xlnet = [xlnet_tokenizer.encode_plus(t, max_length= 120, padding= "max_length", truncation= True,
                                           add_special_tokens= True) for t in X_train_dff] # 
X_test_xlnet = [xlnet_tokenizer.encode_plus(t, max_length= 120, padding= "max_length", truncation= True,
                                          add_special_tokens= True) for t in X_test_dff]

In [ ]:
X_train_xlnet = np.array([inp["input_ids"] for inp in X_train_xlnet])
X_test_xlnet = np.array([inp["input_ids"] for inp in X_test_xlnet])

Fonction pour plotter les métriques.

In [ ]:
def dl_metrics(history, metric_1, metric_2, metric_3):
    fig = plt.figure(figsize=(10, 45))
    # Historique d'accuracy
    plt.subplot(3, 1, 1)
    plt.plot(history.history[metric_1], color="g")
    plt.plot(history.history["val_" + metric_1],
              linestyle= "--", color="orange")
    plt.title("Métrique: " + metric_1, fontsize=18)
    plt.ylabel(metric_1)
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc= "upper left")
    # Historique de f1_m
    plt.subplot(3, 1, 2)
    plt.plot(history.history[metric_2], color="g")
    plt.plot(history.history["val_" + metric_2],
              linestyle= "--", color= "orange")
    plt.title("Métrique: " + metric_2, fontsize=18)
    plt.ylabel(metric_2)
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc= "upper left")
    # Historique de loss
    plt.subplot(3, 1, 3)
    plt.plot(history.history[metric_3], color="g")
    plt.plot(history.history["val_" + metric_3],
              linestyle= "--", color= "orange")
    plt.title("Métrique: " + metric_3, fontsize=18)
    plt.ylabel(metric_3)
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc= "upper left")
    plt.show()

XLNet modèle

***L'entraînement avec les poids du modèle fixés ainsi l'optimisation des hyperparamètres sont mis en commentaires pour des raisons de ressources. Les résultats peuvent être vu sur la Version #11 de ce kernel.***

In [ ]:
clear_session()
gc.collect()
# cuda.select_device(0)
# cuda.close()

In [ ]:
f1 = F1Score(1, average= "weighted")

In [ ]:
# def create_xlnet_keyword_model(xlnet_base):
#     word_inputs = tf.keras.Input(shape=(120,), name='word_inputs', dtype='int32') # X_train_xlnet.shape[1]

#     xlnet = TFXLNetModel.from_pretrained(xlnet_base)
    
#     for layer in xlnet.layers:
#         layer.trainable = False
    
#     xlnet_encodings = xlnet(word_inputs)[0]

#     # CLASSIFICATION HEAD 
#     # Collect last step from last hidden state (CLS)
#     doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
#     # Apply dropout for regularization
# #     doc_encoding = tf.keras.layers.Dense(2048, activation= "relu")(doc_encoding)
#     doc_encoding = tf.keras.layers.Dense(1024, activation= "relu")(doc_encoding)
# #     doc_encoding = tf.keras.layers.Dropout(.2)(doc_encoding)
# #     doc_encoding = tf.keras.layers.Dense(512, activation= "relu")(doc_encoding)
#     doc_encoding = tf.keras.layers.Dropout(.2)(doc_encoding)
#     # Final output 
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

#     # Compile model
#     model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 2e-5), loss='binary_crossentropy', 
#         metrics=['accuracy', f1])

#     return model

In [ ]:
# xlnet_model_pft = create_xlnet_keyword_model(xlnet_pretrained_model)

In [ ]:
# xlnet_model_pft.summary()

In [ ]:
# for layer in xlnet_model_pft.layers[-25:]:
#     print(layer)

In [ ]:
def warmup(epoch, lr):
    """Used for increasing the learning rate slowly, this tends to achieve better convergence.
    However, as we are finetuning for few epoch it's not crucial.
    """
    return max(lr + 1e-6, 2e-5)

In [ ]:
# checkpoint_filepath = "./xlnet_pft_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]


In [ ]:
# xlnet_history = xlnet_model_pft.fit(X_train_xlnet, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(xlnet_history, "accuracy", "f1_score", "loss")

In [ ]:
# xlnet_model_pft.evaluate(X_test_xlnet, y_test_dff, batch_size= 16)

Essayons d'optimiser les paramètres de la couche Dense.

In [ ]:
# def model_builder(hp):
#     word_inputs = tf.keras.Input(shape=(120,), name='word_inputs', dtype='int32') # X_train_xlnet.shape[1]

#     xlnet = TFXLNetModel.from_pretrained(xlnet_pretrained_model)
    
#     for layer in xlnet.layers:
#         layer.trainable = False
    
#     xlnet_encodings = xlnet(word_inputs)[0]

#     # CLASSIFICATION HEAD 
#     # Collect last step from last hidden state (CLS)
#     doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
#     # Apply dropout for regularization

#     # Tune dense units
#     hp_units = hp.Int('dense_units',
#                       min_value=512,
#                       max_value=2048,
#                       step=128,
#                       default=1024)
    
#     doc_encoding = tf.keras.layers.Dense(hp_units, activation= "relu")(doc_encoding)
#     doc_encoding = tf.keras.layers.Dropout(.2)(doc_encoding)
#     # Final output 
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

#     # Compile model
#     model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 2e-5), loss='binary_crossentropy', 
#         metrics=['accuracy', f1])

#     return model

In [ ]:
# import keras_tuner as kt

# # Création du tuner de kerastuner
# tuner = kt.RandomSearch(
#     model_builder, 
#     objective='val_accuracy',
#     max_trials=5)

# # Réglage du early stopping
# stop_early = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy',
#     patience=5)

In [ ]:
# # Recherche des meilleurs paramètres
# tuner.search(X_train_xlnet, y_train_dff, batch_size=16, validation_split= 0.2,
#     epochs=10, callbacks=[stop_early])

# # Affichage des meilleurs paramètres
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"Meilleur Dense units : {best_hps.get('dense_units')}")

Meilleur hyperparamètre couche Dense: units = 1792

In [ ]:
# checkpoint_filepath = "./xlnet_hypermodel_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]


In [ ]:
# hypermodel_xlnet = tuner.hypermodel.build(best_hps)
# xlnet_history = hypermodel_xlnet.fit(X_train_xlnet, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(xlnet_history, "accuracy", "f1_score", "loss")

In [ ]:
# hypermodel_xlnet.evaluate(X_test_xlnet, y_test_dff, batch_size= 16)

Voyons les résultats avec du fine tuning.

***Ces derniers ont aussi été commentés pour une question de ressources. Les résultats sont visibles dans la Version #12.***

In [ ]:
# def create_xlnet_keyword_model(xlnet_base):
#     word_inputs = tf.keras.Input(shape=(120,), name='word_inputs', dtype='int32') # X_train_xlnet.shape[1]

#     xlnet = TFXLNetModel.from_pretrained(xlnet_base)
    
#     for layer in xlnet.layers:
#         layer.trainable = True
    
#     xlnet_encodings = xlnet(word_inputs)[0]

#     # CLASSIFICATION HEAD 
#     # Collect last step from last hidden state (CLS)
#     doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
#     # Apply dropout for regularization
# #     doc_encoding = tf.keras.layers.Dense(2048, activation= "relu")(doc_encoding)
#     doc_encoding = tf.keras.layers.Dense(1792, activation= "relu")(doc_encoding)
# #     doc_encoding = tf.keras.layers.Dropout(.2)(doc_encoding)
# #     doc_encoding = tf.keras.layers.Dense(512, activation= "relu")(doc_encoding)
#     doc_encoding = tf.keras.layers.Dropout(.2)(doc_encoding)
#     # Final output 
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

#     # Compile model
#     model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 2e-5), loss='binary_crossentropy', 
#         metrics=['accuracy', f1])

#     return model

In [ ]:
# xlnet_model = create_xlnet_keyword_model(xlnet_pretrained_model)
# xlnet_model.summary()

In [ ]:
# checkpoint_filepath = "./xlnet_model_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]


In [ ]:
# xlnet_history = xlnet_model.fit(X_train_xlnet, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(xlnet_history, "accuracy", "f1_score", "loss")

In [ ]:
# xlnet_model.evaluate(X_test_xlnet, y_test_dff, batch_size= 16)

Résultats des différents essais avec XLNet:

Entraînement:

| -- | xlnet_pft | hypermodel_xlnet | xlnet_model |
| -- | -- | -- | -- |
| F1 score | 0.6000 | 0.6000 | 0.6000 |
| Accuracy | 0.5681 | 0.5521 | 0.5207 |
| Loss | 1.1190 | 1.1068 | 0.8238 |
| Epoch | 7 | 3 | 1 |

Évaluation:

| -- | xlnet_pft | hypermodel_xlnet | xlnet_model |
| -- | -- | -- | -- |
| F1 score | 0.5976 | 0.5976 | 0.5976 |
| Accuracy | 0.6362 | 0.6474 | 0.5739 |
| Loss | 0.6888 | 0.6810 | 0.7510 |

Le modèle avec les hyperparamètres optimisés obtient une accuracy légèrement inférieure (-2.8%) qu'avant l'optimisation. Pour ce qui est de l'évaluation, il obtient une accuray légèrement supérieure (1.8%) cette fois. Je vais donc privilégier le modèle optimisé si je choisis XLNet.

# RoBERTa

Le modèle "riccardode/tweets_disaster" ne peut pas être utilisé.

J'ai trouvé "vinai/bertweet-large" qui peut convenir. Ce modèle fonctionne avec l'algorithme RoBERTa.


In [ ]:
# Modèle RoBERTa pré-entraîné
roberta_pretrained_model = "vinai/bertweet-large"

# Chargement du tokenizer avec les poids du modèle choisi
roberta_tokenizer = RobertaTokenizer.from_pretrained(roberta_pretrained_model)



In [ ]:
MAX_LEN = 90

def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(" ".join(text))
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
X_train_roberta = roberta_encode(X_train_dff, roberta_tokenizer)
X_test_roberta = roberta_encode(X_test_dff, roberta_tokenizer)


***Les entraînements sans et avec optimisations sont mis en commentaires pour une question de ressources. Les résultats sont visibles sur la version #13.***

In [ ]:
def build_model(n_categories, pretrained_model):
#     with strategy.scope():
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaModel.from_pretrained(pretrained_model)
    
    for layer in roberta_model.layers:
        layer.trainable = False
    
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy', f1])

    return model

In [ ]:
roberta_model_pft = build_model(1, roberta_pretrained_model)
roberta_model_pft.summary()

In [ ]:
checkpoint_filepath = "./roberta_pft_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]

In [ ]:
# roberta_history = roberta_model_pft.fit(X_train_roberta, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(roberta_history, "accuracy", "f1_score", "loss")

In [ ]:
# roberta_model_pft.evaluate(X_test_roberta, y_test_dff, batch_size= 16)

Optimisation des hyperparamètres

In [ ]:
# def model_builder(hp):
# #     with strategy.scope():
#     input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
#     input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
#     input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

#     # Import RoBERTa model from HuggingFace
#     roberta_model = TFRobertaModel.from_pretrained(roberta_pretrained_model)
    
#     for layer in roberta_model.layers:
#         layer.trainable = False
    
#     x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

#     # Huggingface transformers have multiple outputs, embeddings are the first one,
#     # so let's slice out the first position
#     x = x[0]

#     x = tf.keras.layers.Dropout(0.1)(x)
#     x = tf.keras.layers.Flatten()(x)
    
#         # Tune dense units
#     hp_units = hp.Int('dense_units',
#                       min_value=32,
#                       max_value=516,
#                       step=32,
#                       default=256)
    
#     x = tf.keras.layers.Dense(hp_units, activation='relu')(x)
#     x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

#     model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#         loss='binary_crossentropy',
#         metrics=['accuracy', f1])

#     return model


In [ ]:
# import keras_tuner as kt

# # Création du tuner de kerastuner
# tuner = kt.RandomSearch(
#     model_builder, 
#     objective='val_accuracy',
#     max_trials=5)

# # Réglage du early stopping
# stop_early = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy',
#     patience=5)

In [ ]:
# # Recherche des meilleurs paramètres
# tuner.search(X_train_roberta, y_train_dff, batch_size=16, validation_split= 0.2,
#     epochs=10, callbacks=[stop_early])

# # Affichage des meilleurs paramètres
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"Meilleur Dense units : {best_hps.get('dense_units')}")

<!-- Meilleur hyperparamètre pour la couche Dense: units = 224 -->

In [ ]:
# checkpoint_filepath = "./roberta_hypermodel_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]


In [ ]:
# hypermodel_roberta = tuner.hypermodel.build(best_hps)
# roberta_history = hypermodel_roberta.fit(X_train_roberta, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(roberta_history, "accuracy", "f1_score", "loss")

In [ ]:
# hypermodel_roberta.evaluate(X_test_roberta, y_test_dff, batch_size= 16)

Voyons les résultats de RoBERTa après un fine tuning.

***Ces résultats ont été mis en commentaires pour les mêmes raisons que précédemment. Ils sont visibles dans la version #13.***

In [ ]:
# def build_model(n_categories, pretrained_model):
# #     with strategy.scope():
#     input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
#     input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
#     input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

#     # Import RoBERTa model from HuggingFace
#     roberta_model = TFRobertaModel.from_pretrained(pretrained_model)
    
#     for layer in roberta_model.layers:
#         layer.trainable = True
    
#     x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

#     # Huggingface transformers have multiple outputs, embeddings are the first one,
#     # so let's slice out the first position
#     x = x[0]

#     x = tf.keras.layers.Dropout(0.1)(x)
#     x = tf.keras.layers.Flatten()(x)
#     x = tf.keras.layers.Dense(224, activation='relu')(x)
#     x = tf.keras.layers.Dense(n_categories, activation='sigmoid')(x)

#     model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#         loss='binary_crossentropy',
#         metrics=['accuracy', f1])

#     return model

In [ ]:
# roberta_model = build_model(1, roberta_pretrained_model)
# roberta_model.summary()

In [ ]:
# checkpoint_filepath = "./roberta_model_checkpoint/checkpoint"

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 4, min_delta=0.02, 
#                                      restore_best_weights=True),
#     tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
#     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor= 1e-6, patience=2, 
#                                          verbose=0, mode='auto', 
#                                          min_delta=0.001, cooldown=0, min_lr=1e-6),
#     tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
#                                        monitor='val_accuracy', mode='max', save_best_only=True)
# ]

In [ ]:
# roberta_history = roberta_model.fit(X_train_roberta, y= y_train_dff, epochs= 20, batch_size= 16, 
#                                           validation_split= 0.2, callbacks= callbacks)

In [ ]:
# dl_metrics(roberta_history, "accuracy", "f1_score", "loss")

In [ ]:
# roberta_model.evaluate(X_test_roberta, y_test_dff, batch_size= 16)

Résultats des différents essais avec RoBERTa:

Entraînement:

| -- | roberta_pft | hypermodel_roberta | roberta_model |
| -- | -- | -- | -- |
| F1 score | 0.6000 | 0.6000 | 0.6000 |
| Accuracy | 0.7098 | 0.6765 | 0.5472 |
| Loss | 0.5699 | 0.6104 | 0.7047 |
| Epoch | 3 | 2 | 1 |

Évaluation:

| -- | roberta_pft | hypermodel_roberta | roberta_model |
| -- | -- | -- | -- |
| F1 score | 0.5976 | 0.5976 | 0.5976 |
| Accuracy | 0.6658 | 0.6605 | 0.5739 |
| Loss | 0.6335 | 0.6185 | 0.7041 |

Le modèle avec les hyperparamètres de bases, c'est-à-dire non optimisés, obtient les meilleurs accuracy autant pour la phase d'entraînement que d'évaluation. Je vais donc privilégier le modèle de base si je choisis RoBERTa.

# Comparaison des modèles

Résultats issus des entraînements et évaluations:

Entraînement

| -- | XLNet | RoBERTa |
| -- | -- | -- |
| Epochs | 5 | 3 |
| F1 score | 0.6000 | 0.6000 |
| Accuracy | 0.5521 | 0.7098 |
| Loss | 1.1068 | 0.5699 |

Évaluation

| -- | XLNet | RoBERTa |
| -- | -- | -- |
| F1 score | 0.5976 | 0.5976 |
| Accuracy | 0.6474 | 0.6658 |
| Loss | 0.6810 | 0.6335 |

On peut voir que les F1 scores des deux modèles sont égaux.
L'accuracy de RoBERTa est supérieure de 28.6% à celle de XLNet lors de l'entraînement et de 2.8% lors de l'évaluation.

De plus, RoBERTa est plus rapide à entraîner que XLNet avec respectivement 279ms/step et 444ms/step.

Je vais donc opter pour RoBERTa pour obtenir les prédictions de la compétition.


In [ ]:
roberta_model_pft.load_weights("../input/RoBERTa-checkpoint/checkpoint")

Affichage de la matrice de confusion et du rapport de classification.

In [ ]:
predictions = roberta_model_pft.predict(X_test_roberta)[:, 0]

In [ ]:
predictions

Analyse de la courbe ROC pour obtenir un Recall supérieur à 80% et une Précision ~= 50%.

In [ ]:
from sklearn import metrics

[fpr, tpr, thr] = metrics.roc_curve(y_test_dff, predictions)
plt.plot(fpr, tpr, color='coral', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - specificite', fontsize=14)
plt.ylabel('Sensibilite', fontsize=14)

In [ ]:
# indice du premier seuil pour lequel
# la sensibilité est supérieure à 0.75
idx = np.min(np.where(tpr > 0.95)) 

print("Sensibilité : {:.2f}".format(tpr[idx]))
print("Spécificité : {:.2f}".format(1-fpr[idx]))
print("Seuil : {:.2f}".format(thr[idx]))

In [ ]:
print(metrics.auc(fpr, tpr))

L'aire sous la courbe vaut 0.696 ce qui indique que le classifieur est partiellement aléatoire.

Le seuil est fixé à 0.13.

In [ ]:
preds = np.where(predictions >= 0.5, 1, 0)
preds[:25]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

fig = plt.figure(figsize=(5, 5))
ax = sns.heatmap(confusion_matrix(y_test_dff, preds), annot= True)
ax.set_xlabel("Labels predits", color="g")
ax.set_ylabel("True labels", color="orange")
# ax.xaxis.set_ticklabels(le.classes_, 
#                         rotation='vertical')
# ax.yaxis.set_ticklabels(le.classes_,
#                         rotation='horizontal')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, support = precision_recall_fscore_support(y_test_dff, preds, average= "binary")

print(f"Détails de la matrice de confusion:")
print(f"Précision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {fscore}")
support

In [ ]:
print(classification_report(
    y_test_dff, preds, 
#     target_names= set(y_test_dff)
))

Prédictions finales pour la compétition.

In [ ]:
test_roberta = roberta_encode(test_dff["decoded_joined"], roberta_tokenizer)


In [ ]:
test_dff["target"] = roberta_model_pft.predict(test_roberta)

In [ ]:
test_dff["target"] = test_dff["target"].mask(test_dff["target"] >= 0.5, 1).astype("int")
test_dff["target"] = test_dff["target"].mask(test_dff["target"] < 0.5, 0).astype("int")

In [ ]:
submission = pd.DataFrame(test_dff[["id", "target"]])
submission

In [ ]:
submission.to_csv("submission.csv", index= False)